In [2]:
!pip install langchain

In [3]:
!pip install openai

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf"  #허깅페이스 키 입력

In [ ]:
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

hfEndpoint = HuggingFaceEndpoint(
    repo_id="MLP-KTLim/llama-3-Korean-Bllossom-8B",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
) # type: ignore

llm = ChatHuggingFace(llm=hfEndpoint)


prompt = PromptTemplate(
  input_variables=["country"],
  template= "{country}의 수도는 어디야?",
)

chain = LLMChain(llm=llm, prompt=prompt) #프롬프트와 모델을 체인으로 연결 
chain.run("대한민국")

'대한민국의 수도는 서울입니다.'

In [ ]:
#프롬프트1 정의
prompt1 = PromptTemplate(
    input_variables=['sentence'],
    template="다음 문장을 한글로 번역하세요.\n\n{sentence}"
)

#번역(체인1)에 대한 모델
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="translation")

#프롬프트2 정의
prompt2 = PromptTemplate.from_template(
    "다음 문장을 한 문장으로 요약하세요.\n\n{translation}"
)
#요약(체인2)에 대한 모델
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="summary")



from langchain.chains import SequentialChain
all_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=['sentence'],
    output_variables=['translation','summary'],
)
sentence="""
One limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.
For this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.
"""
all_chain.invoke(sentence)

{'sentence': '\nOne limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.\nFor this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.\n',
 'translation': 'LLMs의 한 가지 제한은 특정 문서나 이메일에 대한 맥락 정보의 부족입니다. 이를 극복하기 위해서는 LLM에 특정 외부 데이터에 접근할 수 있도록 해야 합니다.\n\n이 작업을 위해 먼저 외부 데이터를 문서 로더를 사용하여 로드해야 합니다. LangChain은 다양한 문서 유형(예: PDF, 이메일, 웹사이트, 유튜브 비디오 등)에 맞는 로더를 제공합니다.',
 'summary': 'LLMs의 한 가지 제한은 맥락 정보의 부족이며, 이를 극복하기 위해 외부 데이터에 접근할 수 있도록 하는 것이 필요하며, 이를 위해 LangChain의 문서 로더를 사용하여 다양한 문서 유형을 로드해야 합니다.'}